In [ ]:
from edgar import *
from bs4 import BeautifulSoup

set_identity("example1211.company@access.com")

# company = Company('0001011443')
# filings = company.get_filings(form="13F-HR/A")

def gather_holdings_from_soup(conn, soup):
    """Parse XML soup and return list of Holding objects"""
    holdings = []

    for row in soup.find_all('infoTable'):
        # Extract basic data
        name_of_issuer = row.find('nameOfIssuer').text.strip()
        cusip = row.find('cusip').text.strip()
        title_of_class = row.find('titleOfClass').text.strip()

        # Convert numerical values
        try:
            value = int(float(row.find('value').text.strip()))
            shares_amount = int(float(row.find('sshPrnamt').text.strip()))
        except (ValueError, AttributeError) as e:
            raise ValueError(f"Invalid numerical value in holding data: {e}")

        # Extract enums with validation
        share_type = row.find('sshPrnamtType').text.strip().upper()
        if share_type not in ('SH', 'PRN'):
            raise ValueError(f"Invalid share type: {share_type}")

        investment_discretion = row.find('investmentDiscretion').text.strip().upper()
        if investment_discretion not in ('SOLE', 'DFND', 'OTR'):
            raise ValueError(f"Invalid investment discretion: {investment_discretion}")

        put_call = row.find('putCall')
        put_call = put_call.text.strip().upper() if put_call else "NONE"

        # Extract voting authority
        voting = row.find('votingAuthority')
        try:
            sole = int(voting.find('Sole').text.strip()) if voting else 0
            shared = int(voting.find('Shared').text.strip()) if voting else 0
            none = int(voting.find('None').text.strip()) if voting else 0
        except (AttributeError, ValueError):
            sole = shared = none = 0

        # Create Holding instance
        holding = {
            "shares_or_principal_amount": shares_amount,
            "value": value,
            "title_of_class_id": title_of_class,
            "shares_or_principal_type_id": share_type,
            "put_or_call_id": put_call,
            "investment_discretion_id": investment_discretion,
            "voting_authority_sole": sole,
            "voting_authority_shared": shared,
            "voting_authority_none": none,
            # Additional metadata
            "_issuer_name": name_of_issuer,
            "_issuer_cusip": cusip,
        }


        holdings.append(holding)

    return holdings

# for filing in filings:
#     accession_number = filing.accession_number
#     if accession_number != "0001011443-25-000029":
#         continue

#     for attachment in filing.attachments:
#             if (
#                 attachment.document.endswith('.xml')
#                 and 'primary_doc' not in attachment.document
#             ):
#                 assert attachment.content

#                 soup = BeautifulSoup(attachment.content, 'xml')
#                 holdings = []

#                 holdings = gather_holdings_from_soup(None, soup)
